# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 2

### Introdução

Na parte 1 deste desafio, você realizou o pré-processamento e o treinamento de um modelo a partir de um conjunto de dados base fornecido. Nesta segunda etapa você irá integrar todas as transformações e eventos de treinamento criados anteriormente em uma Pipeline completa para *deploy* no **Watson Machine Learning**!

### Preparação do Notebook

Primeiro realizaremos a instalação do scikit-learn e a importação das mesmas bibliotecas utilizadas anteriormente

In [1]:
# # Primeiro, realizamos a instalação do scikit-learn versão 0.20.0 no Kernel deste notebook:
# !pip install numpy==1.16.4 --upgrade
# !pip install pandas==0.24.2 --upgrade
# !pip install scikit-learn==0.20.3 
### !pip install scikit-learn==0.21 
# !pip install lightgbm
# !pip install xgboost
# !pip install scikit-optimize

In [2]:
# !pip install watson_machine_learning_client

In [3]:
!python --version

Python 3.6.10 :: Anaconda, Inc.


In [4]:
import json
import requests
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
import lightgbm as lgbm

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate

import matplotlib.pyplot as plt

from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
)

In [5]:
# << INSIRA O DATASET COMO UM DATAFRAME PANDAS NESTA CÉLULA >>
import types
import pandas as pd

df_data_1 = pd.read_csv("../Data/dataset_desafio_2.csv")
df_data_1.head()

,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,502375,Márcia Illiglener,0,0,0,0,6.2,5.8,4.6,5.9,0.0,2,4,3,EXATAS
1,397093,Jason Jytereoman Izoimum,0,0,0,0,6.0,6.2,5.2,4.5,1.0,2,4,3,EXATAS
2,915288,Bartolomeu Inácio da Gama,0,0,0,0,7.3,6.7,7.1,7.2,0.0,5,0,3,HUMANAS
3,192652,Fernanda Guedes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,4,4,4,DIFICULDADE
4,949491,Alessandre Borba Gomes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,5,2,5,DIFICULDADE


In [6]:
# # remover outliers

# def remove_outliers(df):
#     df = df.query("NOTA_DE >= 0 and NOTA_DE <= 10")
#     df = df.query("NOTA_EM >= 0 and NOTA_EM <= 10")
#     df = df.query("NOTA_MF >= 0 and NOTA_MF <= 10")
#     df = df.query("NOTA_GO >= 0 and NOTA_GO <= 10")
    
#     return df

# print(df_data_1.shape)
# df_data_1 = remove_outliers(df_data_1)
# print("after removal: ", df_data_1.shape)

### Construção da Pipeline completa para encapsulamento no WML

vamos importar nossos transformers do nosso github

In [7]:
# substitua o link abaixo pelo link do seu repositório git (se for o caso)
!git clone https://github.com/lgabs/sklearn_transforms.git

fatal: destination path 'sklearn_transforms' already exists and is not an empty directory.


In [8]:
!cd sklearn_transforms
!ls -ltr

total 1512
-rw-rw-r-- 1 luan luan  45012 Ago 20 07:55 Feature Engineering-Copy1.ipynb
-rw-rw-r-- 1 luan luan  65306 Ago 20 08:15 Tunning XGBoost + Ft Eng.ipynb
-rw-rw-r-- 1 luan luan 405354 Ago 20 08:50 Feature Engineering.ipynb
-rw-rw-r-- 1 luan luan  32887 Ago 20 22:07 parte-1.ipynb
-rw-rw-r-- 1 luan luan 108994 Ago 21 07:56 fastai.ipynb
-rw-rw-r-- 1 luan luan 562890 Ago 21 08:42 Tunning LGBM.ipynb
drwxrwxr-x 4 luan luan   4096 Ago 21 12:01 catboost_info
-rw-rw-r-- 1 luan luan  57813 Ago 21 12:18 logs.log
-rw-rw-r-- 1 luan luan  27938 Ago 21 13:09 parte-2.ipynb
drwxrwxr-x 5 luan luan   4096 Ago 21 21:04 sklearn_transforms
-rw-rw-r-- 1 luan luan  64419 Ago 21 22:37 sklearn_transforms.zip
-rw-rw-r-- 1 luan luan 113177 Ago 21 22:40 pycaret.ipynb
-rw-rw-r-- 1 luan luan  33198 Ago 21 22:58 1st Submission.ipynb


In [9]:
!zip -r sklearn_transforms.zip sklearn_transforms

updating: sklearn_transforms/ (stored 0%)
updating: sklearn_transforms/setup.py (deflated 46%)
updating: sklearn_transforms/README.md (deflated 15%)
updating: sklearn_transforms/.git/ (stored 0%)
updating: sklearn_transforms/.git/index (deflated 52%)
updating: sklearn_transforms/.git/config (deflated 34%)
updating: sklearn_transforms/.git/description (deflated 14%)
updating: sklearn_transforms/.git/logs/ (stored 0%)
updating: sklearn_transforms/.git/logs/HEAD (deflated 29%)
updating: sklearn_transforms/.git/logs/refs/ (stored 0%)
updating: sklearn_transforms/.git/logs/refs/heads/ (stored 0%)
updating: sklearn_transforms/.git/logs/refs/heads/master (deflated 29%)
updating: sklearn_transforms/.git/logs/refs/remotes/ (stored 0%)
updating: sklearn_transforms/.git/logs/refs/remotes/origin/ (stored 0%)
updating: sklearn_transforms/.git/logs/refs/remotes/origin/HEAD (deflated 29%)
updating: sklearn_transforms/.git/branches/ (stored 0%)
updating: sklearn_transforms/.git/packed-refs (deflated 8

In [10]:
!ls

1st Submission.ipynb		 parte-2.ipynb
catboost_info			 pycaret.ipynb
fastai.ipynb			 sklearn_transforms
Feature Engineering-Copy1.ipynb  sklearn_transforms.zip
Feature Engineering.ipynb	 Tunning LGBM.ipynb
logs.log			 Tunning XGBoost + Ft Eng.ipynb
parte-1.ipynb


##### Agora podemos instalar nossa bilioteca e importá-la!

In [11]:
!pip install sklearn_transforms.zip

Processing ./sklearn_transforms.zip
  Created wheel for my-custom-sklearn-transforms: filename=my_custom_sklearn_transforms-1.0-py3-none-any.whl size=2222 sha256=85305e9b8e00133f822b11ccd9492b77e69ee01d0c03a9cde0349f829a757422
  Stored in directory: /tmp/pip-ephem-wheel-cache-4trkkl3u/wheels/b9/90/07/52d6becfa1d68c0bb6d447d451f79c3f428aa1a18167f90dbb
Successfully built my-custom-sklearn-transforms
  Attempting uninstall: my-custom-sklearn-transforms
    Found existing installation: my-custom-sklearn-transforms 1.0
    Uninstalling my-custom-sklearn-transforms-1.0:
      Successfully uninstalled my-custom-sklearn-transforms-1.0


In [12]:
from my_custom_sklearn_transforms.sklearn_transformers import DropColumns, AddFeatures

#### Declarando a Pipeline

Após a importação das transformações personalizadas como um pacote Python, podemos partir para a declaração da nossa Pipeline.

O processo é bem semelhante ao realizado na primeira etapa, porém com algumas diferenças importantes, então preste bem atenção!

A Pipeline exemplo possui três estágios: 

    - remover a coluna "NOME"
    - imputar "zeros" em todos os valores faltantes
    - inserir os dados pré-processados como entrada em um modelo treinado
    
Relembrando, a entrada desta Pipeline será o conjunto cru de dados fornecido exceto a coluna "LABELS" (variável-alvo a ser determinada pelo modelo).

Teremos então 17 valores de entrada **na PIPELINE** (no modelo serão 16 entradas, pois a coluna NAME será removida no primeiro estágio após a transformação DropColumn).

    MATRICULA       - número de quatro algarismos único para cada estudante
    NOME            - nome completo do estudante
    FALTAS_DE       - número de faltas na disciplina de ``Direito Empresarial``
    FALTAS_EM       - número de faltas na disciplina de ``Empreendedorismo``
    FALTAS_MF       - número de faltas na disciplina de ``Matemática Financeira``
    MEDIA_DE        - média simples das notas do aluno na disciplina de ``Direito Empresarial`` (0-10)
    MEDIA_EM        - média simples das notas do aluno na disciplina de ``Empreendedorismo`` (0-10)
    MEDIA_MF        - média simples das notas do aluno na disciplina de ``Matemática Financeira`` (0-10)
    HRS_ESTUDO_DE   - horas de estudo particular na disciplina de ``Direito Empresarial``
    HRS_ESTUDO_EM   - horas de estudo particular na disciplina de ``Empreendedorismo``
    HRS_ESTUDO_MF   - horas de estudo particular na disciplina de ``Matemática Financeira``
    REPROVACOES_DE  - número de reprovações na disciplina de ``Direito Empresarial``
    REPROVACOES_EM  - número de reprovações na disciplina de ``Empreendedorismo``
    REPROVACOES_MF  - número de reprovações na disciplina de ``Matemática Financeira``
    LIVROS_TEXTO    - quantidade de livros e textos acessados pelo aluno no sistema da universidade
    AULAS_AO_VIVO   - horas de aulas ao vivo presenciadas pelo aluno (total em todas as disciplinas)
    EXERCICIOS      - número de exercícios realizados pelo estudante (total em todas as disciplinas) no sistema da universidade

A saída da Pipeline será um valor estimado para a coluna "LABELS".

In [13]:
# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data_1[features]
y = df_data_1[target]

**ATENÇÃO!!**

A célula acima, embora muito parecida com a definição de features na primeira etapa deste desafio, possui uma grande diferença!

Nela está presente a coluna "NOME" como uma feature! Isso ocorre pois neste caso essas são as entradas da *PIPELINE*, e não do modelo.

In [14]:
# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)

Na célula abaixo é realizada a declaração de um objeto **Pipeline** do scikit-learn, onde é declarado o parâmetro *steps*, que nada mais é do que uma lista com as etapas da nossa pipeline:

    'remove_cols'     - transformação personalizada DropColumns
    'imputer'         - transformação embutida do scikit-learn para imputação de valores faltantes
    'dtc'             - um classificador via árvore de decisão
    
Note que passamos como passos as transformadas instanciadas anteriormente, sob nome `rm_columns` e `si`.

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MATRICULA       20000 non-null  int64  
 1   NOME            20000 non-null  object 
 2   REPROVACOES_DE  20000 non-null  int64  
 3   REPROVACOES_EM  20000 non-null  int64  
 4   REPROVACOES_MF  20000 non-null  int64  
 5   REPROVACOES_GO  20000 non-null  int64  
 6   NOTA_DE         20000 non-null  float64
 7   NOTA_EM         20000 non-null  float64
 8   NOTA_MF         20000 non-null  float64
 9   NOTA_GO         16284 non-null  float64
 10  INGLES          16372 non-null  float64
 11  H_AULA_PRES     20000 non-null  int64  
 12  TAREFAS_ONLINE  20000 non-null  int64  
 13  FALTAS          20000 non-null  int64  
dtypes: float64(5), int64(8), object(1)
memory usage: 2.1+ MB


In [16]:
print("train size: ", len(X_train))
print("train size: ",len(X_test))

train size:  14000
train size:  6000


### LGBM sem tunning

In [17]:
X_train.head()

,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS
15979,373711,Giuseppe Mineiro Júnior,0,0,0,0,5.6,5.3,5.5,NaN,NaN,2,3,5
7847,581227,Alana Evelyn Maciel,1,1,1,1,0.0,0.0,0.0,0.0,1.0,3,2,3
16536,663504,André Ushizuwo,0,0,0,0,4.5,5.6,5.2,4.8,1.0,4,1,6
8589,647585,Bárbara Érica do Rio,0,0,0,0,6.1,4.7,4.5,5.6,NaN,6,5,3
19964,812616,Breno Kauê de Castro Vunonucru,0,0,0,0,6.5,6.2,6.5,NaN,1.0,5,2,3


In [18]:
# Criação de uma Transform personalizada ``DropColumns``

rm_columns = DropColumns(
    columns=["NOME"]
)

# Criação de um objeto ``SimpleImputer``

si = SimpleImputer(
    missing_values=np.nan,  # os valores faltantes são do tipo ``np.nan`` (padrão Pandas)
    strategy='constant',  # a estratégia escolhida é a alteração do valor faltante por uma constante
    fill_value=-99999,  # a constante que será usada para preenchimento dos valores faltantes é um int64=0.
#     strategy = "most_frequent",
    verbose=0,
    copy=True
)

# já tunados previamente por CV
best_params = {'num_leaves': 16,
 'learning_rate': 0.04255941370614545,
 'min_child_samples': 55,
 'subsample': 0.048101030476692466,
 'colsample_bytree': 0.6439559798028359}

# clf = lgbm.LGBMClassifier(random_state=42)
# clf = XGBClassifier()
# clf = RandomForestClassifier()

clf.set_params(**best_params)

pipeline = Pipeline(
    steps=[
        ('remove_cols', rm_columns),
#         ('add_features', AddFeatures()),
        ('imputer', si),
        ('clf', clf),
    ]
)

# Inicialização da Pipeline (pré-processamento e realização do treinamento do modelo)
pipeline.fit(X_train, y_train.values.ravel())

# Realização de teste cego no modelo criado
from sklearn.metrics import accuracy_score

# Acurácia alcançada pela árvore de decisão
y_pred = pipeline.predict(X_test)
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred), 4)))

NameError: name 'clf' is not defined

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

### Cross Validation

In [ ]:
cv_pipeline = Pipeline(
    steps=[
        ('remove_cols', rm_columns),
        ('add_features', AddFeatures()),
        ('imputer', si),
        ('clf', lgbm.LGBMClassifier(**best_params)),
    ]
)

np.mean(cross_val_score(cv_pipeline, X, y, cv=3, n_jobs=1, scoring="accuracy"))

### Encapsulando uma Pipeline personalizada no Watson Machine Learning

#### Estabelecendo conexão entre o cliente Python do WML e a sua instância do serviço na nuvem

In [ ]:
# !pip install watson_machine_learning_client

In [ ]:
# !pip install pyspark

In [ ]:
# import joblib

In [ ]:
# Biblioteca Python com implementação de um cliente HTTP para a API do WML
from watson_machine_learning_client import WatsonMachineLearningAPIClient

As próximas células irão realizar o deploy da pipeline declarada neste notebook no WML. Só prossiga se você já está satisfeito com seu modelo e acha que já é a hora de fazer o deploy da sua solução.

Cole as credenciais de sua instância do Watson Machine Learning na variável na célula abaixo.

É importante que a variável que contém os valores tenha o nome de ``wml_credentials`` para que as próximas células deste notebook executem corretamente.

In [ ]:
wml_credentials = {
  "apikey": "gbh6pGJCkZi3v2f4KioL5F1nzEsOgVeMWQ6xJGUxR6Lk",
  "iam_apikey_description": "Auto-generated for key 9a39090a-4f8c-4e33-84fa-a95563f8ed94",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/552ebdafcabd49bb8fac0272e72b3d7d::serviceid:ServiceId-8e6bfd2c-d190-4858-bfe3-023f5f782f4c",
  "instance_id": "3e0edd1b-a9ab-4a75-9ee3-53568a566e37",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [ ]:
# Instanciando um objeto cliente do Watson Machine Learning a partir das credenciais fornecidas
clientWML = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# Extraindo detalhes da sua instância do Watson Machine Learning

instance_details = clientWML.service_instance.get_details()
print(json.dumps(instance_details, indent=4))

**ATENÇÃO!!**

Fique atento para os limites de consumo de sua instância do Watson Machine Learning!

Caso você expire a camada grátis, não será possível avaliar seu modelo (pois é necessária a realização de algumas chamadas de API que consomem predições!)

#### Listando todos os artefatos armazenados no seu WML

Para listar todos os artefatos armazenados em seu Watson Machine Learning, você pode usar a seguinte função:

    clientWML.repository.list()

In [ ]:
# Listando todos os artefatos atualmente armazenados na sua instância do WML

# artifact_guid = "0600007f-d06c-4446-b439-6f1c2bcfb685"
# clientWML.repository.delete(artifact_guid)

clientWML.repository.list()

No plano LITE do Watson Machine Learning só é permitido o deploy de um único modelo por vez. Se for o caso de você já possuir um modelo online na sua instância, você pode apagá-lo utilizando o método clientWML.repository.delete():

    artifact_guid = "359c8951-d2fe-4063-8706-cc06b32d5e0d"
    clientWML.repository.delete(artifact_guid)

#### Criando uma nova definição de pacote Python personalizado no WML

O primeiro passo para realizar seu deploy é armazenar o código das transformações personalizadas criadas por você.

Para essa etapa precisamos apenas do arquivo .zip do pacote criado (que já possuimos carregado no Kernel!)

In [ ]:
!ls

In [ ]:
# Definição de metadados do nosso pacote com as Transforms personalizadas
pkg_meta = {
    clientWML.runtimes.LibraryMetaNames.NAME: "my_custom_sklearn_transform_1",
    clientWML.runtimes.LibraryMetaNames.DESCRIPTION: "A custom sklearn transform",
    clientWML.runtimes.LibraryMetaNames.FILEPATH: "sklearn_transforms.zip",  # Note que estamos utilizando o .zip criado anteriormente!
    clientWML.runtimes.LibraryMetaNames.VERSION: "1.0",
    clientWML.runtimes.LibraryMetaNames.PLATFORM: { "name": "python", "versions": ["3.6"] }
}
custom_package_details = clientWML.runtimes.store_library( pkg_meta )
custom_package_uid = clientWML.runtimes.get_library_uid( custom_package_details )



In [ ]:
print("\n Lista de artefatos de runtime armazenados no WML:")
clientWML.repository.list()

#### Criando uma nova definição de runtime Python personalizado no WML

O segundo passo é armazenar uma definição de runtime Python para utilizar a nossa biblioteca personalizada.

Isso pode ser feito da seguinte forma:

In [ ]:
!python --version

In [ ]:
runtime_meta = {
    clientWML.runtimes.ConfigurationMetaNames.NAME: "my_custom_wml_runtime_1",
    clientWML.runtimes.ConfigurationMetaNames.DESCRIPTION: "A Python runtime with custom sklearn Transforms",
    clientWML.runtimes.ConfigurationMetaNames.PLATFORM: {
        "name": "python",
        "version": "3.6"
    },
    clientWML.runtimes.ConfigurationMetaNames.LIBRARIES_UIDS: [ custom_package_uid ]
}
runtime_details = clientWML.runtimes.store( runtime_meta )
custom_runtime_uid = clientWML.runtimes.get_uid( runtime_details )

print("\n Detalhes do runtime armazenado:")
print(json.dumps(runtime_details, indent=4))

In [ ]:
custom_runtime_uid = "85771af1-2444-469a-b050-d4b169a6f282"
clientWML.runtimes.get_details(custom_runtime_uid)

In [ ]:
# Listando todos runtimes armazenados no seu WML:
# help(clientWML.runtimes.list)

d6fe4db0-4f89-41dd-a8e1-460b4adbdc1a could not be found."

#### Criando uma nova definição de Pipeline personalizada no WML

Finalmente iremos criar uma definição (metadados) para a nossa Pipeline ser hospedada no WML.

Definimos como parâmetros um nome para o artefato e o ID do runtime criado anteriormente.

In [ ]:
model_meta = {
    clientWML.repository.ModelMetaNames.NAME: 'desafio-2-mbtc2020-pipeline-1',
    clientWML.repository.ModelMetaNames.DESCRIPTION: "my pipeline for submission",
    clientWML.repository.ModelMetaNames.RUNTIME_UID: custom_runtime_uid
}

Em seguida chamamos o método para armazenar a nova definição:

In [ ]:
pipeline

In [ ]:
# Função para armazenar uma definição de Pipeline no WML
stored_model_details = clientWML.repository.store_model(
    model=pipeline,  # `my_pipeline` é a variável criada anteriormente e contém nossa Pipeline já treinada :)
    meta_props=model_meta,  # Metadados definidos na célula anterior
    training_data=None  # Não altere esse parâmetro
)


In [ ]:
print("\n Lista de artefatos armazenados no WML:")
clientWML.repository.list()

# Detalhes do modelo hospedado no Watson Machine Learning
print("\n Metadados do modelo armazenado:")
print(json.dumps(stored_model_details, indent=4))

#### Realizando o deployment do seu modelo para consumo imediato por outras aplicações

In [ ]:
# O deployment do modelo é finalmente realizado por meio do método ``deployments.create()``

model_deployment_details = clientWML.deployments.create(
    artifact_uid=stored_model_details["metadata"]["guid"],  # Não altere esse parâmetro
    name="desafio-2-mbtc2020-deployment-1",
    description="Solução do desafio 2 - MBTC",
    asynchronous=False,  # Não altere esse parâmetro
    deployment_type='online',  # Não altere esse parâmetro
    deployment_format='Core ML',  # Não altere esse parâmetro
    meta_props=model_meta  # Não altere esse parâmetro
)

#### Testando um modelo hospedado no Watson Machine Learning

In [ ]:
# Recuperando a URL endpoint do modelo hospedado na célula anterior

model_endpoint_url = clientWML.deployments.get_scoring_url(model_deployment_details)
print("A URL de chamada da sua API é: {}".format(model_endpoint_url))

In [ ]:
# Detalhes do deployment realizado

deployment_details = clientWML.deployments.get_details(
    deployment_uid=model_deployment_details["metadata"]["guid"]  # esse é o ID do seu deployment!
)

print("Metadados do deployment realizado: \n")
print(json.dumps(deployment_details, indent=4))

In [ ]:
scoring_payload = {
    'fields': [
        "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
        "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
        "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    ],
    'values': [
        [
            513949,"Marli Quésia de Oliveira",1,1,1,1,4.3,4.0,3.1,4.9,0,3,4,3,
        ]
    ]
}

print("\n Payload de dados a ser classificada:")
print(json.dumps(scoring_payload, indent=4))

In [ ]:
result = clientWML.deployments.score(
    model_endpoint_url,
    scoring_payload
)

print("\n Resultados:")
print(json.dumps(result, indent=4))

<hr>

## Parabéns! 

Se tudo foi executado sem erros, você já tem um classificador baseado em machine learning encapsulado como uma API REST!

Para testar a sua solução integrada com um assistente virtual e realizar a submissão, acesse a página:

https://uninassau.maratona.dev

Você irá precisar da endpoint url do seu modelo e das credenciais do WML :)